# "제목 !!"
> "요약!! "

- toc:true
- branch: master
- badges: true
- comments: true
- author: In Chan
- categories: [jupyter, deep learning]

In [1]:
from torch.utils.data import random_split
import torch
# import Engine
from utils import *
import utils
import math
from torch.autograd import Function
import os
from torch.utils.tensorboard import SummaryWriter
from model import Extractor, Classifier
from Engine import Engine
import time
import optuna


In [3]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    log, logclose = create_logger(log_filename=log_filename,display=False)
    log(f'Trial Number = {trial.number}')
    n_layers = trial.suggest_int("n_layers", 1, 5)

    layers = []

    in_ch = 2
    in_size=66
    Activation_name_generator = trial.suggest_categorical("Activate Function for Generator", ["ReLU", "LeakyReLU", "ELU","Tanh"])
    Activation_generator = getattr(nn, Activation_name_generator)
    
    for i in range(5):
        out_ch = [int(96),int(256),int(384),int(384),int(256)]
        padding= trial.suggest_int("padding{}".format(i), 0, 5)
        kernel= trial.suggest_int("kernel{}".format(i), 1, 6)
        stride= trial.suggest_int("stride{}".format(i), 1, 5)
        Max_stride = trial.suggest_int("Max_stride{}".format(i), low=1, high=2)
        # Dilation = trial.suggest_int("Dilation{}".format(i),1, 3)

        layers.append(nn.Conv1d(in_ch, out_ch[i], kernel, stride, padding))
        layers.append(nn.BatchNorm1d(out_ch[i]))
        layers.append(Activation_generator())
        if i==1 or 2 or 5 :
            layers.append(nn.MaxPool1d(2,Max_stride))
        
        in_ch=out_ch[i]

        in_size=((in_size-kernel+2*padding)/stride +1)
        # in_size=((in_size+2*padding-Dilation*(kernel-1))/stride +1)

        if in_size.__class__ == float :
            in_size = float(math.floor(in_size))
        
        if i == 1 or 2 or 5 : 
            if Max_stride == 2 : 
                in_size=(in_size-2)/2+1
                if in_size.__class__ == float :
                    in_size = float(math.floor(in_size))
            if Max_stride == 1 : 
                in_size=(in_size-2)+1
                if in_size.__class__ == float :
                    in_size = float(math.floor(in_size))

        
        if in_size <=10 :
            break
    layers.append(nn.Flatten())  

    
    # layers.append(nn.Linear(int(in_size*out_ch[i]), num_fc_node, bias=True))
    Feature_extractor = nn.Sequential(*layers)
    
    ## Classifier
    class_layers = trial.suggest_int("n_layers", 1, 5)
    layers =[]
    
    Activation_name_classifier = trial.suggest_categorical("Activate Function for Classifier", ["ReLU", "LeakyReLU", "ELU","Tanh"])
    Activation_classifier = getattr(nn, Activation_name_classifier)
    
    node_in_size = int(in_size*out_ch[i])
    for i in range(class_layers):
        out_size= trial.suggest_int("cl_out_size{}".format(i), 10, 1000)
        layers.append(nn.Linear(node_in_size, out_size, bias=True))
        layers.append(Activation_classifier())
        node_in_size = out_size

    layers.append(nn.Linear(out_size,6))
    Classifier_layer = nn.Sequential(*layers)

    class CNN(torch.nn.Module):
        
        def __init__(self, Feature_extractor ):
            super(CNN, self).__init__()
            self.Feature_extractor = Feature_extractor
            
        
        def forward(self, x):
            features = self.Feature_extractor(x)
            return features

    Feature_extractor = CNN(Feature_extractor)

    class DNN(torch.nn.Module):        
        def __init__(self, Classifier ):
            super(DNN, self).__init__()
            self.Classifier = Classifier
        def forward(self, x):
            Pred = self.Classifier(x)
            return Pred

    Feature_extractor = CNN(Feature_extractor)
    Classifier1 = DNN(Classifier_layer)
    Classifier2 = DNN(Classifier_layer)

    log('#####################################################################')
    log('############################### Model ################################')
    for key, value in trial.params.items():
        log("{}: {}".format(key, value))
    log('##############################################################################')
    log('############################### Hyper Prameters #################################')
    log(f'Generator Activation : {Activation_name_generator}')
    log(f'Classifier Activation : {Activation_name_classifier}')
    logclose()

    return Feature_extractor , Classifier1, Classifier2

In [4]:
def objective(trial):
    # log, logclose = create_logger(log_filename=log_filename)
    


    #############################################################
    DEVICE = 'cuda'
    EPOCH = 30 
    Feature_Extractor, Classifier1, Classifier2 = define_model(trial)
    Feature_Extractor = Feature_Extractor.to(DEVICE)
    Classifier1 = Classifier1.to(DEVICE)
    Classifier2 = Classifier2.to(DEVICE)

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "Adagrad","Adadelta"])

    lr_E = trial.suggest_float("lr", 1e-7, 1e-3, log=True)
    lr_C = trial.suggest_float("lr2", 1e-7, 1e-3, log=True)
    
    optimizer_E = getattr(optim, optimizer_name)(Feature_Extractor.parameters(), lr=lr_E)
    optimizer_C1 = getattr(optim, optimizer_name)(Classifier1.parameters(), lr=lr_C)
    optimizer_C2 = getattr(optim, optimizer_name)(Classifier2.parameters(), lr=lr_C)
    
    Criterion = nn.CrossEntropyLoss()
    
    scheduler_name = trial.suggest_categorical("scheduler", ["LambdaLR", "StepLR", "CosineAnnealingLR",'None'])
    if scheduler_name == "LambdaLR" :
        scheduler_E = optim.lr_scheduler.LambdaLR(optimizer = optimizer_E,lr_lambda = lambda epoch:0.95 **epoch)
        scheduler_C1 = optim.lr_scheduler.LambdaLR(optimizer = optimizer_C1,lr_lambda = lambda epoch:0.95 **epoch)
        scheduler_C2 = optim.lr_scheduler.LambdaLR(optimizer = optimizer_C2,lr_lambda = lambda epoch:0.95 **epoch)
    if scheduler_name == "StepLR" :
        scheduler_E = optim.lr_scheduler.StepLR(optimizer = optimizer_E,step_size = 6 ,gamma=0.75)
        scheduler_C1 = optim.lr_scheduler.StepLR(optimizer = optimizer_C1,step_size = 6 ,gamma=0.75)
        scheduler_C2 = optim.lr_scheduler.StepLR(optimizer = optimizer_C2,step_size = 6 ,gamma=0.75)
    if scheduler_name == "CosineAnnealingLR" :
        scheduler_E = optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer_E,T_max=5)
        scheduler_C1 = optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer_C1,T_max=5)
        scheduler_C2 = optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer_C2,T_max=5)
    if scheduler_name == "None" :
        scheduler_E = optim.lr_scheduler.LambdaLR(optimizer = optimizer_E,lr_lambda = lambda epoch:1 **epoch)
        scheduler_C1 = optim.lr_scheduler.LambdaLR(optimizer = optimizer_C1,lr_lambda = lambda epoch:1 **epoch)
        scheduler_C2 = optim.lr_scheduler.LambdaLR(optimizer = optimizer_C2,lr_lambda = lambda epoch:1 **epoch)
        
    N = trial.suggest_int("Training Number of Generator", 1, 10) # Training Number of Generator 
    #############################################################

    Best_Loss = np.inf
    Best_Accuracy_10 = 0
    Best_Accuracy_30 = 0

    log, logclose = create_logger(log_filename=log_filename,display=False)
    log(f'Generator Learning Rate = {lr_E}')
    log(f'Classifier Learning Rate = {lr_C}')
    log(f'Optimizer Name : {optimizer_name}')
    log(f'Scheduler Name : {scheduler_name}')
    log(f'Training Number of Generator : {N}')
    log('############################### Hyper Prameters #################################')
    log('##############################################################################')
    log('\n')
    # print('Start Learning')
    Eng = Engine(optimizer_E,optimizer_C1,optimizer_C2,Feature_Extractor,Classifier1,Classifier2,DEVICE)
    for epoch in range(EPOCH) : 
        Loss_A , Loss_B, Loss_C = Eng.train(source_loader, target_loader,N)
        ACC1_10 , ACC2_10, Test_Loss_1_10, Test_Loss_2_10 = Eng.test(test_loader_10)
        ACC1_30 , ACC2_30, Test_Loss_1_30, Test_Loss_2_30 = Eng.test(test_loader_30)
        
        scheduler_E.step()
        scheduler_C1.step()
        scheduler_C2.step()

        log, logclose = create_logger(log_filename=log_filename)
        log(f'============= Epoch_{epoch} =============')
        log(f'Loss A = {Loss_A:0.4f} , Loss B = {Loss_B:0.4f} , Loss C = {Loss_C:0.4f}')
        log(f'Accuracy 1 - 10% SL = {ACC1_10*100:0.2f}, Accuracy 2 - 10% SL = {ACC2_10*100:0.2f} ')
        log(f'Test Loss 1 - 10% SL = {Test_Loss_1_10:0.4f}, Test Loss 2 - 10% SL = {Test_Loss_2_10:0.4f} ')
        log(f'Accuracy 1 - 30% SL = {ACC1_30*100:0.2f}, Accuracy 2 - 30% SL = {ACC2_30*100:0.2f} ')
        log(f'Test Loss 1 - 30% SL = {Test_Loss_1_30:0.4f}, Test Loss 2 - 30% SL = {Test_Loss_2_30:0.4f} ')

        if max(ACC1_10,ACC2_10) > Best_Accuracy_10 : 
            Best_Accuracy_10 = max(ACC1_10,ACC2_10)
            log(f'--------------------------------Best Accuracy_10 = {Best_Accuracy_10 * 100}--------------------------------')
            ACC_10 = [ACC1_10,ACC2_10]
            ACC_30 = [ACC1_30,ACC2_30]
            index = ACC_10.index(Best_Accuracy_10)
            Best_Accuracy_30 = ACC_30[index]
            log(f'--------------------------------Best Accuracy_30 = {Best_Accuracy_30 * 100}--------------------------------')

        logclose()
        trial.report(Best_Accuracy_10,Best_Accuracy_30, epoch)
            # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
################################################################
    return Best_Accuracy_10,Best_Accuracy_30

In [7]:
name = time.asctime(time.localtime(time.time()))
name = name.replace(' ','_').replace(':','_')
log_dir = f'./log/{name}'
os.makedirs(log_dir, exist_ok=True)
log_filename = os.path.join(log_dir, 'train.log')

# Data Load
folds = make_folds(5)
S_train_data , S_train_label , _ , _ = source_Load_data(folds)
_ , _ , T_test_data , T_test_label = target_Load_data(folds)

test_data_10 , test_label_10 =test_Load_data_10(folds)
test_data_30 , test_label_30 =test_Load_data_30(folds)

source_set = utils.CustomDataset(data=S_train_data, label = S_train_label)
source_loader=DataLoader(source_set,batch_size=32,shuffle=True,drop_last=False)

target_set = utils.CustomDataset(data=T_test_data, label = T_test_label)
target_loader = DataLoader(target_set,batch_size=32,shuffle=True,drop_last=False)

test_set_10 = utils.CustomDataset(data=test_data_10, label = test_label_10)
test_loader_10 = DataLoader(test_set_10,batch_size=32,shuffle=True,drop_last=False)

test_set_30 = utils.CustomDataset(data=test_data_30, label = test_label_30)
test_loader_30 = DataLoader(test_set_30,batch_size=32,shuffle=True,drop_last=False)

# study = optuna.create_study(direction=["maximize","maximize"],sampler=optuna.samplers.TPESampler())
study = optuna.create_study(direction=["maximize","maximize"])
# study = optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=5000)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

ValueError: Please set either 'minimize' or 'maximize' to direction. You can also set the corresponding `StudyDirection` member.

In [ ]:
plot_optimization_history(study)
plot_intermediate_values(study)
plot_parallel_coordinate(study)
plot_parallel_coordinate(study, params=["bagging_freq", "bagging_fraction"])
plot_contour(study)
plot_contour(study, params=["bagging_freq", "bagging_fraction"])
plot_slice(study)
plot_slice(study, params=["bagging_freq", "bagging_fraction"])
plot_param_importances(study)
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)
plot_edf(study)